<a href="https://colab.research.google.com/github/AhmedShiny/gpt-researcher/blob/master/IdeaValidation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tavily-python langchain-google-genai crewai crewai_tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.2/76.2 kB 884.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.9/163.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.4/337.4 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.0/774.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
#Loading tavily and running it
from google.colab import userdata
from tavily import TavilyClient

tavily = TavilyClient(api_key=userdata.get('TAVILY_API_KEY'))
# For advanced search:
response = tavily.search(query="what is the best product to sell for dropshipping", search_depth="advanced")
# Get the search results as context to pass an LLM:
context = [{"url": obj["url"], "content": obj["content"]} for obj in response["results"]]

In [3]:
#loading gemini model
from langchain_google_genai import ChatGoogleGenerativeAI
import os
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
llm = ChatGoogleGenerativeAI(model="gemini-pro")
result = llm.invoke("tell a joke")
print(result.content)

What do you call a boomerang that doesn't come back?

A stick.


In [5]:
!pip install embedchain

  Using cached embedchain-0.1.113-py3-none-any.whl (202 kB)
  Using cached langchain-0.1.20-py3-none-any.whl (1.0 MB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
  Using cached langchain_community-0.0.38-py3-none-any.whl (2.0 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 5.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of langchain-cohere to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [10]:
from crewai_tools import tool

@tool("Researcher")
def Researcher(question: str) -> str:
    """
    This function performs a search using Tavily and returns the results as a list of dictionaries.
    """
    # For advanced search:
    response = tavily.search(query=f"I'm making a {question} You are given a question .You have to thorughly research this idea,like what challenges will happen and how to overcome it etc.Search various sources.Conduct Market research and see trends etc", search_depth="advanced")
    # Get the search results as context to pass an LLM:
    context = [{"url": obj["url"], "content": obj["content"]} for obj in response["results"]]
    return context

In [11]:
from crewai import Agent

researcher = Agent(
  role='Senior Researcher',
  goal='Search what is asked using the help of tavily to validate the given idea ',
  verbose=True,
  memory=True,
  backstory=(
    """You are an expert in validating Ideas especially Saas ideas if they are good or not and how they can be imporved.
      You search about the ideas using the Tavily .Doing Market Search,Searching for relevant surveys etc anything that can help in validating the idea """
  ),
  llm=llm,
  tools=[Researcher],
  allow_delegation=True
)


In [8]:
from crewai import Task

research_task = Task(
  description=(
    "Ideas is: {topic}."
    "You are given an idea,you have to validate wheter it's good or not and how it can be improved.Don't be general give specific answers"
    "Give answer using the tavily"
    "try to stick to the question"
    "Answer should be in human readable form and organized"

    "When adding link add a refrence from where the information is from at the bottom"
  ),
  expected_output='Answer returned with the help of taviy in human readble form.Also return a link at the bottom for what source you used from tavily.The answer should be a report and should provide solutions',
  tools=[Researcher],
  agent=researcher,
)


In [13]:
from crewai import Crew
crew = Crew(
            agents=[researcher],
            tasks=[research_task],
            verbose=2,
            max_iterations=5
        )

result = crew.kickoff(inputs={'topic': 'Im making a saas that helps in validate Saas Ideas'})

print(result)

 [2024-07-03 04:28:29][DEBUG]: == Working Agent: Senior Researcher
 [2024-07-03 04:28:29][INFO]: == Starting Task:  Im making a saas that helps in validate Saas Ideas.You are given an idea,you have to validate wheter it's good or not and how it can be improvedGive answer using the tavilytry to stick to the questionAnswer should be in human readable form and organizedWhen adding link add a refrence from where the information is from at the bottom


> Entering new CrewAgentExecutor chain...
Action: Researcher
Action Input: {"question": "What are the key factors to consider when evaluating a SaaS idea?"} 

[{'url': 'https://www.linkedin.com/pulse/comprehensive-guide-saas-vendor-evaluation-making', 'content': 'Choosing the right SaaS vendor is crucial for the success of your business. In this comprehensive guide, we provide step-by-step instructions and key considerations for evaluating SaaS vendors. By ...'}, {'url': 'https://connect.comptia.org/content/guides/msp-checklist-factors-to-con